In [1]:
# Load package and set directory path
import pandas as pd

dir_path = 'I:/Data_for_practice/NA Sci-Tech Nexus/Data/'

In [2]:
# Load cpc data
cpc_info = pd.read_csv(dir_path+"uspto_cpc_long_utility.csv")
cpc_info.head()

,patent_id,cpc_subclass,cpc_group,cpc_subgroup,cpcagg
0,3930271,A63B,A63B71,A63B71/146,A
1,3930272,A47D,A47D7,A47D7/02,A
2,3930272,A47D,A47D9,A47D9/012,A
3,3930272,Y10T,Y10T403,Y10T403/32451,Y
4,3930273,A61G,A61G7,A61G7/0507,A


In [3]:
# Load citation data
citations = pd.read_csv(dir_path+"citations_utility_domestic24.csv")
citations.head()

,citing_patent_id,cited_patent_id
0,10000000,5093563
1,10000000,5751830
2,10000001,7804268
3,10000001,9022767
4,10000001,9090016


In [4]:
# Load patent data and add year
patents = pd.read_csv(dir_path+"patent_utility_dates_usonly.csv")
patents['filing_year'] = patents['filing_date'].astype(str).str[:4].astype(int)
patents.head()

,patent_id,grant_date,filing_date,usweight,filing_year
0,3930273,1976-01-06,1975-01-10,1.0,1975
1,3930274,1976-01-06,1974-10-11,1.0,1974
2,3930275,1976-01-06,1975-03-31,1.0,1975
3,3930276,1976-01-06,1972-08-07,1.0,1972
4,3930277,1976-01-06,1974-08-21,1.0,1974


In [5]:
# Create citation table with filing year of citing patent
citing_patents = patents[['patent_id', 'filing_year']].rename(columns={'patent_id': 'citing_patent_id', 'filing_year': 'citing_fyear'})
citations = citations.merge(citing_patents, on='citing_patent_id', how='inner')
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear
0,10000000,5093563,2015
1,10000000,5751830,2015
2,10000007,1404819,2016
3,10000007,2085447,2016
4,10000007,2241645,2016


In [6]:
# Add cited patent filing year
cited_patents = patents[['patent_id', 'filing_year']].rename(columns={'patent_id': 'cited_patent_id', 'filing_year': 'cited_fyear'})
citations = citations.merge(cited_patents, on='cited_patent_id', how='inner')
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear
0,10000000,5093563,2015,1989
1,10000000,5751830,2015,1995
2,10000007,4342216,2016,1980
3,10000007,4604890,2016,1985
4,10000007,4689857,2016,1987


In [7]:
# Create citation lag
citations['fcitelag'] = citations['citing_fyear'] - citations['cited_fyear']
citations = citations[citations['fcitelag'] >= 0]
citations.head()

,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fcitelag
0,10000000,5093563,2015,1989,26
1,10000000,5751830,2015,1995,20
2,10000007,4342216,2016,1980,36
3,10000007,4604890,2016,1985,31
4,10000007,4689857,2016,1987,29


In [8]:
# Create CPC 1digit class and merge with patent data
cpc_info['cpcagg'] = cpc_info['cpc_subclass'].astype(str).str[0]
patents_cpc = patents.merge(cpc_info, on='patent_id', how='inner')
patents_cpc.head()

,patent_id,grant_date,filing_date,usweight,filing_year,cpc_subclass,cpc_group,cpc_subgroup,cpcagg
0,3930273,1976-01-06,1975-01-10,1.0,1975,A61G,A61G7,A61G7/0507,A
1,3930273,1976-01-06,1975-01-10,1.0,1975,A61G,A61G7,A61G7/0509,A
2,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B7,B63B7/085,B
3,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B34,B63B34/00,B
4,3930274,1976-01-06,1974-10-11,1.0,1974,B63B,B63B35,B63B35/34,B


In [9]:
# Create focal patent set (year 2000)
focal_patents = patents_cpc[patents_cpc['filing_year'] == 2000]
focal_patents.head()

,patent_id,grant_date,filing_date,usweight,filing_year,cpc_subclass,cpc_group,cpc_subgroup,cpcagg
5130337,6076844,2000-06-20,2000-01-07,1.000000,2000,B60R,B60R3,B60R3/00,B
5147039,6083063,2000-07-04,2000-01-13,1.000000,2000,B63H,B63H11,B63H11/01,B
5183071,6095256,2000-08-01,2000-01-19,1.000000,2000,B44B,B44B5,B44B5/0033,B
5185308,6096107,2000-08-01,2000-01-03,0.666667,2000,B24D,B24D3,B24D3/06,B
5185309,6096107,2000-08-01,2000-01-03,0.666667,2000,B24D,B24D11,B24D11/005,B


In [10]:
# focal_citations: Prior patents cited by focal year patents
focal_citations = focal_patents.merge(citations, left_on='patent_id', right_on='citing_patent_id', how='inner')
focal_citations['focal_patent_id'] = focal_citations['patent_id']
focal_citations = focal_citations[['focal_patent_id','citing_patent_id', 'cited_patent_id', 'citing_fyear', 'cited_fyear']]
focal_citations['fit'] = 1 
focal_citations.head()

,focal_patent_id,citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fit
0,6076844,6076844,4102432,2000,1977,1
1,6076844,6076844,5054799,2000,1990,1
2,6076844,6076844,5593167,2000,1994,1
3,6083063,6083063,4373919,2000,1980,1
4,6083063,6083063,4538996,2000,1983,1


In [11]:
# newer_citations: Follow-on patent recite the prior patent.
newer_citations = citations[(citations['citing_fyear'] > 2000) & (citations['citing_fyear'] < 2006)]
newer_citations = newer_citations.merge(focal_citations[['cited_patent_id']], on='cited_patent_id', how='inner')
newer_citations.rename(columns={'citing_patent_id': 'newer_citing_patent_id'}, inplace=True)
newer_citations = newer_citations.drop_duplicates()
newer_citations['bit'] = 1
newer_citations.head()

,newer_citing_patent_id,cited_patent_id,citing_fyear,cited_fyear,fcitelag,bit
0,10000788,4098876,2002,1976,26,1
27,10000788,4115535,2002,1977,25,1
31,10000788,4177253,2002,1977,25,1
65,10000788,4436826,2002,1981,21,1
95,10000788,4454233,2002,1981,21,1


In [15]:
# Merge newer_citations and focal_citations
# newer_citing_patent_id: Prior patent's Follow-on patent
# cited_patent_id: Focal Patent's Prior patent
# focal_patent_id: Focal patent
# bit: 1 if newer_citing_patent_id cites cited_patent_id
# fit: 1 if focal_patent_id cites cited_patent_id
cd_index_data = newer_citations[['newer_citing_patent_id','cited_patent_id','bit']].merge(focal_citations[['focal_patent_id','cited_patent_id','fit']],
                                                                                    left_on='cited_patent_id', 
                                                                                    right_on='cited_patent_id', how='left')
cd_index_data.fillna(0, inplace=True)
cd_index_data.head()

,newer_citing_patent_id,cited_patent_id,bit,focal_patent_id,fit
0,10000788,4098876,1,6440682,1
1,10000788,4098876,1,6440682,1
2,10000788,4098876,1,6440682,1
3,10000788,4098876,1,6440682,1
4,10000788,4098876,1,6440682,1


In [13]:
cd_index_data['fitbit'] = cd_index_data['fit'] * cd_index_data['bit']
cd_index_data['cdindex'] = -2 * cd_index_data['fitbit'] + cd_index_data['fit']
cd_index_data.head()

,newer_citing_patent_id,cited_patent_id,bit,focal_patent_id,fit,fitbit,cdindex
0,10000788,4098876,1,6440682,1,1,-1
1,10000788,4098876,1,6440682,1,1,-1
2,10000788,4098876,1,6440682,1,1,-1
3,10000788,4098876,1,6440682,1,1,-1
4,10000788,4098876,1,6440682,1,1,-1


In [14]:
cd_summary = cd_index_data.groupby('focal_patent_id')['cdindex'].sum().reset_index()
cd_summary['cdindex'] /= cd_summary['focal_patent_id'].map(cd_summary['focal_patent_id'].value_counts())
cd_summary

,focal_patent_id,cdindex
0,6076844,-13.0
1,6083063,-29.0
2,6095256,-15.0
3,6096107,-450.0
4,6098530,-17.0
...,...,...
100819,10275780,-76302.0
100820,10293056,-3640.0
100821,10361802,-3453494.0
100822,10387952,-4714.0
